# 11. VULNERABILITIES

1. [x] введение
2. [x] VulBERTa
3. [x] упражнение
4. [x] ссылки

# 1. Введение

*Уязвимость (vulnerability)* --- недостаток в системе, который может нарушить eё целостность. Уязвимость можно рассматривать в качесте одного из типов дефектов

### Как возникают?

Уязвимость может быть результатом:
- ошибок программирования
- недостатков, допущенных при проектировании системы
- ненадежных паролей
- вирусов
- других вредоносных программ, скриптовых и SQL-инъекций.

### Чем опасны?

Обычно уязвимость позволяет атакующему "обмануть" приложение:
- выполнить непредусмотренные создателем действия или
- заставить приложение совершить действие, на которое у того не должно быть прав.

### Примеры?

```
var express = require('express')

var app = express()
const Sequelize = require('sequelize');
const sequelize = new Sequelize('database', 'username', 'password', {
  dialect: 'sqlite',
  storage: 'data/juiceshop.sqlite'
});

app.post('/login', function (req, res) {
    sequelize.query('SELECT * FROM Products WHERE name LIKE ' +  req.body.username);
  })
```

Ещё примеры: [Vulnerable-Code-Snippets](https://github.com/snoopysecurity/Vulnerable-Code-Snippets)


### Как обнаружить?

Для выявления уязвимостей проводятся *пентесты* (тестирование на проникновение).

Появляются подходы с применением машинного обучения.

# 2. VulBERTa

## 2.1 Статья

[Hanif Maffeis - VulBERTa: Simplified Source Code Pre-Training for Vulnerability Detection 2022](https://arxiv.org/abs/2205.12424) --- использование архитектуры Трансформер для обнаружения уязвимостей в C/C++ коде.

## 2.2 Подход

Используется модель RoBERTa с задачей MLM.
На выходе токены коды преобразуются в эмбеддинги длины 768.
Далее эмбеддинги используются для в качестве вход для нейронной сети, которая выполняет классификацию.

Общая схема:

![](./res/11_vulberta_overview.png)

## 2.3 Токенизация

Общая схема:
![](./res/11_vulberta_tokenization.png)

### Парсер

- На входе: код.
- На выходе: промежуточные токены.

Как работает парсер?
0. работаем на уровне функий --- разбиваем код на отдельные функции
1. убираем комментарии из каждой функции с помощью регулярных выражений
2. разбиваем на промежуточные токены (учитываем синтаксическую структуру кода) с помощью библиотеки [Clang](https://clang.llvm.org/)

![](./res/11_vulberta_parser.png)


### Токенизатор

- На входе: промежуточные токены.
- На выходе: токены (идентификаторы).

Токенизатор основан на [BPE](http://ethen8181.github.io/machine-learning/deep_learning/subword/bpe.html) (byte pair encoding). Кроме того, в словарь токенов добавляем дополнительные токены --- ключевые слова из языка программирования, пунктуацию, названия некоторых функций. Таким образом, мы хотим обеспечить, чтобы ключевые слова не дробились и информация о исходном коде лучше сохранялась.
Вместе со специальными BPE-токенами мы имеем следующие 451 предопределённый токен:

![](./res/11_vulberta_tokens.png)

Остальные токены определяются в результате алгоритма BPE.

После того, как определены токены, происходит кодирование токенов --- получаем тензоры длины 512 (1024 для файнтюнинга), использую токен `<pad>`.

## 2.4 Pre-training

Предобучение --- это обучение модили [RoBERTa-base](https://huggingface.co/roberta-base) на задаче MLM (masked language modelling).
Отличия BERT и RoBERTa:
- в BERT маскирование выполняется только один раз во время подготовки данных. Например, каждое предложение маскируется 10 различными способами. Во время обучения модель увидит только эти 10 вариантов каждого предложения
- в RoBERTa маскировка выполняется во время обучения. Следовательно, каждый раз, когда предложение включается в batch, оно маскируется, и, следовательно, количество потенциально различных замаскированных версий каждого предложения неограничено


Для обучения нужны данные. Здесь использовалось два датасета GitHub и Draper.

### GitHub 

- 1101075 функций на языках C/C++
- 1060 репозитория
- для сбора данных использовались: [GitHub API](https://docs.github.com/en/rest) и [PyGitHub](https://github.com/PyGithub/PyGithub)
- репозитории выбирались на основе количества звёзд
- для извлечения функций исползовалась библиотека [Joern](https://joern.io/)

### Draper

- 1274366 функций на языках C/C++
- есть разметка, выполненная с помощью инструментов статического анализа (потенциальные уязвимости)
- датасет [опубликован](https://osf.io/d45bw)

### Обучение

- Google Compute Engine (GCP) VMs
- 48 vCPUs
- 240GB RAM
- 2 NVIDIA Tesla A100 40GB GPUs
- 72--96 часов на эксперимент
- 500000 шагов (обновлений градиента)

## 2.5 Fine-tuning

Здесь реализовано два подхода: VulBERTa-MLP и VulBERTa-CNN.

![](./res/11_vulberta_finetuning.png)

### VulBERTa-MLP

Эмбеддинги из предобученного RoBERTa отправляются в MLP, где происходит классификация на два класса: содержит ли код уязвимость или нет.
- полносвязный слой (размер 768)
- выходной слой (размер 2)
- бинарная классификация

![](./res/11_vulberta_mlp.png)

Обучаем только MLP, несколько эпох.

### VulBERTa-CNN

В этом подходе используется Text Convolutional Neural Network ([TextCNN](https://github.com/delldu/TextCNN)).

![](./res/11_vulberta_textcnn.jpg)

- одномерная свёрточная сеть
- слой max-pooling
- выходы конкатенируются и вытягиваются
- полносвязный слой (размер 256)
- полносвязный слой (размер 128)
- выходной слой для классификации

![](./res/11_vulberta_cnn.png)

Обучение происходит быстрее, чем для VulBERT-MLP.

### Данные для файнтюнинга

1. [Vuldeepecker](https://github.com/CGCL-codes/VulDeePecker)
2. [Draper](https://osf.io/d45bw)
3. [REVEAL](https://arxiv.org/abs/2009.07235)
4. [muVuldeepecker](https://github.com/muVulDeePecker/muVulDeePecker)
5. [Devign](https://arxiv.org/abs/1909.03496)
6. [D2A](https://github.com/IBM/D2A)

### Обучение

- a machine with 48 cores Intel Xeon Silver CPU
- 292GB RAM
- 2 NVIDIA GTX TITAN Xp GPU (12GB of video memory)
- 10 эпох
- 5--10 часов

## 2.6 Evaluation

![](./res/11_vulberta_results.png)

![](./res/11_vulberta_benchmarks.png)

## 2.7 Реализация

github: https://github.com/ICL-ml4csec/VulBERTa


## 3. Упражнение

Использовать VulBERTa для поиска уязвимостей в любом из репозиториев.

# 4. Ссылки

- [What is a Vulnerability? Definition + Examples](https://www.upguard.com/blog/vulnerability)
- https://ulearn.me/Course/Hackerdom/Temy_kursa_2bc58982-19aa-49ef-bae2-8aaa5d13b967
- Software Vulnerability Detection Using Deep Neural Networks: A Survey (2020), Proceedings of the IEEE, Lin, Guanjun, et al.
- https://github.com/saltudelft/ml4se#bugvulnerability-detection
- I. Rosenberg, G. Sicard, and E. David, “End-to-end deep neural networks and transfer learning for automatic analysis of nation-state malware,” Entropy, vol. 20, no. 5, p. 390, 2018.
- L. Chen, S. Sultana, and R. Sahita, “Henet: A deep learning approach on intel processor trace for effective exploit detection,” in 2018 IEEE security and privacy workshops (SPW), 2018, pp. 109--115.
- Zhang et al - COMBO Pre-training representations of binary code using contrastive learning 2022
- [Hanif Maffeis - VulBERTa: Simplified Source Code Pre-Training for Vulnerability Detection](https://arxiv.org/abs/2205.12424)